# 第一部分 划分词频


## 1 读取《红楼梦》的文本数据，使用codecs包，它可以先通过设置文件的编码，对文件进行读入

In [10]:
import codecs
file = codecs.open("HLM.txt", 'r', 'utf-8')
content = file.read()
file.close()
#print content

## 2 引用jieba分词, 添加自定义词典

jieba.load_userdict(file_name) # file_name为自定义词典的路径
示例：
云计算 
李小福 
创新办 
之前： 李小福 / 是 / 创新 / 办 / 主任 / 也 / 是 / 云 / 计算 / 方面 / 的 / 专家 / 
加载自定义词库后： 李小福 / 是 / 创新办 / 主任 / 也 / 是 / 云计算 / 方面 / 的 / 专家 / 
"通过用户自定义词典来增强歧义纠错能力" --- https://github.com/fxsjy/jieba/issues/14

import simplejson
print simplejson.dumps(dict, encoding="UTF-8", ensure_ascii=False)

In [11]:
# -*- coding:utf-8 -*- 
import jieba
#import simplejson
jieba.load_userdict("HLMwords.txt")
segments = []
segs = jieba.cut(content)
for seg in segs:
    if len(seg)>1:
        segments.append(seg)
#print segments
#print simplejson.dumps(segments, encoding="UTF-8", ensure_ascii=False)

## 3 引入pandas（依赖于numpy）的 DataFrame（数据框）  DataFrame 是一个表格型的数据结构
    A   B   C   .   .
1   3   4   5   2   3
2   1   2   3   2   5
3   4   2   1   6   3
.   3   3   2   2   3
.   2   2   3   3   2
.   1   2   4   3   1

In [7]:
示例：
>>> data = {'state':['Ohino','Ohino','Ohino','Nevada','Nevada'],
        'year':[2000,2001,2002,2001,2002],
        'pop':[1.5,1.7,3.6,2.4,2.9]}
>>> df = DataFrame(data)
>>> df
   pop   state  year
0  1.5   Ohino  2000
1  1.7   Ohino  2001
2  3.6   Ohino  2002
3  2.4  Nevada  2001
4  2.9  Nevada  2002

[5 rows x 3 columns]

NameError: name 'DataFrame' is not defined

In [12]:
import numpy
import pandas
segmentDF = pandas.DataFrame({'segment':segments})
#print segmentDF

In [13]:
stopwords = pandas.read_csv(u"StopwordsCN.txt",encoding='utf-8',index_col=False,quoting=3,sep="\t",names=['stopword'])
segmentDF = segmentDF[~segmentDF.segment.isin(stopwords.stopword)]
#print segmentDF

In [16]:

stopwords = pandas.read_table(u"StopwordsCN.txt",encoding='utf-8',index_col=False,quoting=3,sep="\t",names=['stopword'])
segmentDF = segmentDF[~segmentDF.segment.isin(stopwords.stopword)]
wyStopWords = pandas.Series([
		  # 42 个文言虚词 
		  '之', '其', '或', '亦', '方', '于', '即', '皆', '因', '仍', '故', 
		  '尚', '呢', '了', '的', '着', '一', '不', '乃', '呀', '吗', '咧', 
		  '啊', '把', '让', '向', '往', '是', '在', '越', '再', '更', '比', 
		  '很', '偏', '别', '好', '可', '便', '就', '但', '儿', 
		  # 高频副词 
		  '又', '也', '都', '要', 
		  # 高频代词 
		  '这', '那', '你', '我', '他',
		  #高频动词
		  '来', '去', '道', '笑', '说',
		  #空格
		  ' ', '','\r\n'
		]);
segmentDF = segmentDF[~segmentDF.segment.isin(wyStopWords)]

# pandas聚合和分组运算
segStat = segmentDF.groupby(by=["segment"])["segment"].agg({"计数":numpy.size}).reset_index().sort_values(by=["计数"],ascending=False);
			

#print segStat.head(42)